앙상블 학습(Ensemble Learning)

In [1]:
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()

In [2]:
from sklearn.preprocessing import StandardScaler
cancer_std = StandardScaler().fit_transform(cancer.data)

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    cancer_std, cancer.target, stratify=cancer.target, test_size=0.2, random_state=2023
)

앙상블 학습(Ensemble Learning) 중 보팅(Voting)은 투표를 통해 최종 예측 결과를 결정하는 방식이며, 서로 다른 유형의 알고리즘을 가진 분류기가 같은 데이터셋을 학습하여 도출한 결과들을 결합한다는 점에서 배깅(Bagging)과 구분된다.
앙상블의 기본 알고리즘으로 일반적으로 사용되는 것은 결정 트리이고 앙상블 방식으로는 크게 보팅, 배깅, 부스팅으로 나뉩니다.

1. Voting 방식.

*** Voting 의 유형
- 하드 보팅(Hard Voting)
 : 예측한 결과값들 중 다수의 분류기가 결정한 예측 결과값을 최종 봍팅 결과값으로 선정.

- 소프트 보팅(Soft Voting)
 : 각 분류기별 레이블 값 결정 확률을 평균 낸 확률이 가장 높은 레이블 값을 최종 보팅 결과값으로 선정.
   일반적으로 이 방식이 성능이 더 좋아 자주 사용.

In [4]:
# Hard Voting_로지스틱 회귀, 서포트 벡터 머신, K 최근접 이웃
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [5]:
lrc = LogisticRegression(random_state=2023)
svc = SVC(random_state=2023)
knn = KNeighborsClassifier(n_neighbors=5)

In [6]:
# 앙상블 분류기_voting방식: hard(기본값)
from sklearn.ensemble import VotingClassifier
voc = VotingClassifier( estimators=[('LRC', lrc), ('SVC', svc),('KNN', knn)],
                       voting = 'hard')

In [7]:
# 앙상블 학습과 평가
voc.fit(X_train, y_train)
voc.score(X_test, y_test)

0.9298245614035088

In [9]:
# 개별 분류기의 학습과 평가
lrc.fit(X_train, y_train)
svc.fit(X_train, y_train)
knn.fit(X_train, y_train)
lrc.score(X_test, y_test), svc.score(X_test, y_test), knn.score(X_test, y_test)

(0.9473684210526315, 0.9298245614035088, 0.9122807017543859)

In [10]:
# Soft voting-로지스틱회귀, 서포트 벡터머신, K최근접 이웃
svc2 = SVC(probability=True, random_state=2023)
svc2.fit(X_train, y_train)
svc2.predict_proba(X_test[:3])

array([[9.99574375e-01, 4.25625266e-04],
       [5.14249474e-08, 9.99999949e-01],
       [1.65822655e-02, 9.83417734e-01]])

In [11]:
voc2 = VotingClassifier(
    estimators=[('LRC', lrc), ('SVC', svc2), ('KNN', knn)],
    voting='soft'
)
voc2.fit(X_train, y_train)
voc2.score(X_test, y_test)

0.9298245614035088

In [12]:
# 개별 분류기의 학습과 평가
lrc.fit(X_train, y_train)
svc2.fit(X_train, y_train)
knn.fit(X_train, y_train)
lrc.score(X_test, y_test), svc2.score(X_test, y_test), knn.score(X_test, y_test)

(0.9473684210526315, 0.9298245614035088, 0.9122807017543859)

In [15]:
# 분류기 정확도
from sklearn.metrics import accuracy_score
classifiers = [lrc,svc2, knn]
for clf in classifiers:
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    class_name = clf.__class__.__name__
    print('{0}정확도 : {1:.4f}'.format(class_name, accuracy_score(y_test, pred)))

LogisticRegression정확도 : 0.9474
SVC정확도 : 0.9298
KNeighborsClassifier정확도 : 0.9123


GridSearchCV 클래스의 생성자 정리

-estimator : classifier, regressor, pipeline 등 가능

-param_grid : 튜닝을 위해 파라미터, 사용될 파라미터를 dictionary 형태로 만들어서 넣는다.

-scoring : 예측 성능을 측정할 평가 방법을 넣는다. 보통 accuracy 로 지정하여서 정확도로 성능 평가를 한다.

-cv : 교차 검증에서 몇개로 분할되는지 지정한다.

-refit : True가 디폴트로 True로 하면 최적의 하이퍼 파라미터를 찾아서 재학습 시킨다.

In [16]:
# GridSearchCV 적용
from sklearn.model_selection import GridSearchCV
params = {
    'LRC__C': [0.1, 1, 10],
    'SVC__C': [0.1, 1, 10]
}
grid_voc2 = GridSearchCV(voc2, params, scoring='accuracy', cv=5)
grid_voc2.fit(X_train, y_train)
grid_voc2.best_params_

{'LRC__C': 10, 'SVC__C': 1}

In [17]:
grid_voc2.best_estimator_.score(X_test, y_test)

0.9473684210526315

2. Bagging 방식
- 배깅은 같은 알고리즘으로 여러개의 분류기를 만들어서 보팅으로 최종 결정하는 알고리즘이다.
- 대표적인 알고리즘은 랜덤 포레스트이다.


2.1 랜덤 포레스트
 - 랜덤 포레스트는 앙상블 알고리즘 중 비교적 빠른 수행 속도를 가지고 있으며, 다양한 영역에서 높은 예측 선능을 보임.
 - 여러개의 결정 트리 분류기가 전체 학습 데이터에서 배깅 방식으로 각자의 데이터를 샘플링해 개별적으로 학습을 수행한 뒤 최종적으로 모든 분류기가 보팅을 통해 예측 결정을 함.


 - 랜덤 포레스트이 파라미터

 ① n_estimators : 랜덤 포레스트에서 결정 트리의 개수를 지정. 디폴트 값은 10.

 ② max_features : 결정 트리에 사용된 max_features 파라미터와 같다. 디폴트 값은 'sqrt'.

 ③ 결정 트리와 동일하게 과적합 개선을 위해서 max_depth와 min_samples_leaf가 사용 됨.

 랜덤 포레스트도 GridSearchCV를 이용해 최적의 하이퍼 파라미터 값을 찾을 수 있음.


* 학습 데이터를 여러개의 데이터 세트로 중첩되게 분리하는 것을 부트스트래핑(bootstrapping)이라고 함.

In [18]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=2023)
rfc.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 2023,
 'verbose': 0,
 'warm_start': False}

In [19]:
rfc.fit(X_train, y_train)

RandomForestClassifier(random_state=2023)

In [20]:
rfc.score(X_test, y_test)

0.9210526315789473

In [21]:
rfc.predict_proba(X_test[:5])

array([[0.99, 0.01],
       [0.  , 1.  ],
       [0.18, 0.82],
       [0.  , 1.  ],
       [0.02, 0.98]])